# Model pro rozpoznávání různých druhů geometrických útvarů

##### Tento notebook je zaměřen na rozpoznávání geometrických útvarů pomocí modelu MobileNetV2

Nakonec jsem se rozhodla místo Google Colab pracovat v prostředí **Jupyter Notebook** (chtěla bych v něm pracovat i na svém maturitním projektu a ráda bych si na něj zvykla). 

Původně jsem plánovala vytrénovat model pro klasifikaci druhů koktejlů. Tento pokus však nebyl úspěšný – i přes několik experimentů a změn jsem nebyla schopna dosáhnout vyšší validační přesnosti než 30 %. Důvodem byla pravděpodobně vizuální podobnost mezi jednotlivými koktejly (např. tvary sklenic), což model zmátlo, protože se neměl na čem naučit jednoznačné rozdíly.

Rozhodla jsem se tedy přejít na jinou úlohu a stáhla jsem si připravený dataset obsahující obrázky geometrických tvarů (kruhy, čtverce, hvězdy, pentagony, heptagony a trojúhelníky). Abych se vyhnula zavádějícím vzorům, zkontrolovala jsem dataset a ujistila se, že obrázky jsou konzistentní – stejné rozlišení a bez rušivých prvků. Navíc jsem použila augmentaci dat (např. rotace nebo změny velikosti), aby model lépe rozlišoval mezi podobnými tvary.

Pro tuto úlohu jsem po dlouhém zkoušení nakonec zvolila předtrénovaný model **MobileNetV2**, který jsem dále upravila pro klasifikaci těchto šesti kategorií. Model se mi podařilo úspěšně natrénovat a nyní funguje i predikce nových obrázků. Největší problém se objevuje hlavně při rozpoznávání pentagonů, a poté i u heptagonů a čtverců, které si model kvůli jejich podobnosti někdy zaměňuje. Bylo by možné na tomto problému ještě zapracovat, například zlepšením kvality dat nebo přidáním dalších vzorů pro učení.

I přesto jsem s výsledkem spokojená, protože model dokáže úlohu řešit s dobrou přesností a tento notebook slouží jako dokumentace celého procesu.

---

**Struktura notebooku**:
1. Instalace a import potřebných knihoven.
2. Předzpracování dat a vytvoření datového generátoru.
3. Definice modelu MobileNetV2.
4. Trénink modelu a vyhodnocení jeho přesnosti.
5. Predikce na nových obrázcích.

## Instalace knihoven

In [1]:
!pip install tensorflow keras 


In [1]:
dataset_path = './dataset'

In [2]:
!pip install opencv-python 

## Importování knihoven a nastavení konstant

In [3]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Nastavení konstant
IMG_SIZE = 128
BATCH_SIZE = 32

## Předzpracování obrázků

In [4]:
def preprocess_image(image):
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE)) # Změna velikosti obrázku na požadovanou velikost (IMG_SIZE x IMG_SIZE)
    
    # Normalizace obrázku: převede hodnoty pixelů z rozsahu [0, 255] na [0, 1]
    image = image / 255.0  # Normalizace na interval [0, 1]
    return image

## Augmentace obrázků

In [5]:
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2  # Použití 20 % obrázků pro validaci (rozdělení dat na trénovací a validační část)
)

## Příprava trénovacích a validačních dat

In [6]:
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)
validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 9600 images belonging to 6 classes.
Found 2400 images belonging to 6 classes.


## Definice modelu pomocí MobileNetV2

In [7]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False

model = Sequential([
    base_model,
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')  # Změňte počet tříd podle datasetu
])


## Kompilace modelu

In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_128            │ (None, 4, 4, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 20480)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     1,310,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,569,158 (13.62 MB)

 Trainable params: 1,311,174 (5.00 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

## Trénování modelu a nastavení callbacků

In [10]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping

# Předčasné zastavení
early_stopping = EarlyStopping(monitor='val_loss', 
                               patience=5,  # Počet epoch, po kterých dojde k zastavení, pokud nedojde ke zlepšení
                               verbose=1,   # Sledování validační ztráty (loss)
                               mode='min',  # Hledání minimální hodnoty ztráty
                               restore_best_weights=True)  # Obnovení nejlepšího modelu


# Snížení rychlosti učení
lr_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                 patience=3, 
                                 factor=0.5, 
                                 min_lr=1e-6)

# Trénování modelu
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=15, # Počet epoch trénování (měnit!)
    callbacks=[lr_reduction, early_stopping],  # Předání callbacků
)

Epoch 1/15
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.3558 - loss: 1.4524

/opt/conda/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


300/300 ━━━━━━━━━━━━━━━━━━━━ 29s 95ms/step - accuracy: 0.3560 - loss: 1.4521 - val_accuracy: 0.6350 - val_loss: 0.9425 - learning_rate: 0.0010
Epoch 2/15
300/300 ━━━━━━━━━━━━━━━━━━━━ 27s 91ms/step - accuracy: 0.4737 - loss: 1.1720 - val_accuracy: 0.6696 - val_loss: 0.8767 - learning_rate: 0.0010
Epoch 3/15
300/300 ━━━━━━━━━━━━━━━━━━━━ 28s 93ms/step - accuracy: 0.4951 - loss: 1.1471 - val_accuracy: 0.6896 - val_loss: 0.7458 - learning_rate: 0.0010
Epoch 4/15
300/300 ━━━━━━━━━━━━━━━━━━━━ 29s 97ms/step - accuracy: 0.5092 - loss: 1.0945 - val_accuracy: 0.6650 - val_loss: 0.8284 - learning_rate: 0.0010
Epoch 5/15
300/300 ━━━━━━━━━━━━━━━━━━━━ 30s 101ms/step - accuracy: 0.5311 - loss: 1.0461 - val_accuracy: 0.7171 - val_loss: 0.7425 - learning_rate: 0.0010
Epoch 6/15
300/300 ━━━━━━━━━━━━━━━━━━━━ 30s 100ms/step - accuracy: 0.5308 - loss: 1.0463 - val_accuracy: 0.7075 - val_loss: 0.7434 - learning_rate: 0.0010
Epoch 7/15
300/300 ━━━━━━━━━━━━━━━━━━━━ 30s 98ms/step - accuracy: 0.5319 - loss: 1.01

## Vyhodnocení modelu

In [11]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation accuracy: {accuracy*100:.2f}%')

75/75 ━━━━━━━━━━━━━━━━━━━━ 6s 74ms/step - accuracy: 0.7302 - loss: 0.6826
Validation accuracy: 73.17%


## Predikce nového obrázku

In [12]:
def predict_image(image_path):
    image = cv2.imread(image_path)
    image = preprocess_image(image)
    image = np.expand_dims(image, axis=0)
    prediction = model.predict(image)

    print (prediction)
    
    # Vrátí třídu s nejvyšší pravděpodobností
    class_labels = ['circle', 'heptagon', 'pentagon', 'square', 'star', 'triangle']  # Upravit kategorie (pozor na pořadí!)
    predicted_class = np.argmax(prediction)
 
    return class_labels[predicted_class]

In [41]:
result = predict_image("./predikce/unknown4.png")
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
[[9.9796665e-01 2.0334234e-03 5.9530782e-19 4.6548778e-38 0.0000000e+00
  0.0000000e+00]]
circle
